In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset, load_metric

# Load pre-trained model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Load dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

# Initialize ROUGE metric
rouge = load_metric('rouge')

def preprocess(text):
    """Preprocess the input text for the model."""
    return tokenizer(text, return_tensors='pt', max_length=1024, truncation=True, padding='max_length')

def summarize(text):
    """Generate a summary for the input text."""
    inputs = preprocess(text)
    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=150,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def evaluate_summary(prediction, reference):
    """Evaluate the generated summary using ROUGE metrics."""
    results = rouge.compute(predictions=[prediction], references=[reference])
    return results



In [ ]:
# Example usage
if __name__ == "__main__":
    # Sample text and reference summary
    text = """
    Artificial intelligence (AI) is a branch of computer science aimed at building smart machines capable of performing tasks that typically require human intelligence. These tasks include reasoning, problem-solving, understanding natural language, and perception. AI research encompasses various subfields, such as machine learning, where algorithms learn from data, and natural language processing, which deals with the interaction between computers and human language.

AI systems are used in numerous applications, including virtual assistants like Siri and Alexa, self-driving cars, and recommendation systems on platforms like Netflix and Amazon. The development of AI technologies involves the use of large datasets and powerful computational resources. Advances in AI have the potential to revolutionize many industries, but they also raise ethical concerns about privacy, job displacement, and the need for responsible AI practices.
    """
    reference_summary = """
    "Artificial intelligence (AI) involves creating smart machines that perform tasks requiring human-like intelligence, such as problem-solving and understanding language. Key areas include machine learning and natural language processing. AI applications range from virtual assistants to self-driving cars and recommendation systems. While AI holds transformative potential, it also presents ethical challenges."
    """

    # Generate summary
    summary = summarize(text)
    print("Generated Summary:", summary)

    # Evaluate the generated summary
    # evaluation_results = evaluate_summary(summary, reference_summary)
    # print("Evaluation Results:", evaluation_results)